In [ ]:
from datetime import datetime
from pathlib import Path
import importlib
import time
import win32com.client
import os
import warnings
warnings.filterwarnings("ignore")  # Oculta todos los warnings

# Mis modulos
import download_mail_file as dmf
import file_management as fm
import graphic_settings as gs
import send_reports_through_wssp as srtw
import print_management as pm

### Banner

In [ ]:
# import pyfiglet
# print(pyfiglet.figlet_format("Ralenti WSSP"))

banner = r"""
 ____       _            _   _  __        ______ ____  ____  
|  _ \ __ _| | ___ _ __ | |_(_) \ \      / / ___/ ___||  _ \ 
| |_) / _` | |/ _ \ '_ \| __| |  \ \ /\ / /\___ \___ \| |_) |
|  _ < (_| | |  __/ | | | |_| |   \ V  V /  ___) |__) |  __/ 
|_| \_\__,_|_|\___|_| |_|\__|_|    \_/\_/  |____/____/|_|                                                               

    💥 AUTOMATIZADOR DE ENVIO DE REPORTES POR WSSP 💥
"""

print(banner, end='\n\n')
time.sleep(1)

### Configuracion

In [ ]:
# Constantes usadas en el notebook
MAPI = "MAPI" # Messaging Application Programming Interface
DOT = "."
OUTLOOK = "Outlook"
APPLICATION = "Application"

# Diccionarios
outlook_folder_codes = {
    0: 'Calendario',
    1: 'Contactos',
    2: 'Borradores',
    3: 'Diario / Jornal',
    4: 'Notas',
    5: 'Tareas',
    6: 'Bandeja de entrada',
    7: 'Bandeja de salida',
    8: 'Elementos enviados',
    9: 'Elementos eliminados',
    10: 'Bandeja de correo del servidor',
    11: 'Conflictos',
    12: 'Elementos de sincronizacion local',
    13: 'Elementos de sincronizacion (Envio)',
    14: 'Elementos de sincronización (Recibo)',
    15: 'Elementos de sincronización completa',
    16: 'Diario de formularios',
    17: 'Carpeta de búsqueda',
    18: 'Bandeja para reglas cliente',
    19: 'Carpeta de sugerencias de correo',
}
outlook_object_types = {
    "AppointmentItem": 26,
    "MailItem": 43,
    "TaskItem": 46,
    "ContactItem": 48,
    "MeetingItem": 53,
}
parse_locaciones = {
    '06 AYA EL PEDREGAL': 'Pedregal',
    '38 AYA ATICO': 'Atico',
    '40 AYA CHALA': 'Chala',
    '88 AYA CAMANA': 'Camana'
}
months = {
    1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
    7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 
    11: 'Noviembre', 12: 'Diciembre'
}
conductores_ralenti = {
    "name": "Conductores ralenti",
    "mail_subject": "Reporte programado en la base de datos lindley_peru: ACL T2 - Resumen Operativo de Flota (Por Conductor) - AYA",   # Nombre del asunto de correo
    "mail_file_address": "",                                                                                                            # Direccion del archivo del correo
    "mail_sheet_name": "Summary",
    "mail_received_time": "",
    "mail_data_time": "",
    "relevant_columns": [
        'LOCACION',
        'CONDUCTOR',
        'RECORRIDO.',
        'TIEMPO RALENTI.',
    ],
    "reports": [ # sobre 'TIEMPO RALENTI.'
        'CONDUCTOR',
    ]
}

# Listas
locaciones = ['PEDREGAL', 'CHALA', 'CAMANA']
PEDREGAL = 'PEDREGAL'
CAMANA = 'CAMANA'
CHALA = 'CHALA'

# Constantes
project_address = os.getcwd()

### Obtener correos de Outlook

Configuracion de uso de Outlook

In [17]:
# Conectar a Outlook
#outlook_folder_codes = int(input(f'{" ".join(["(" + str(key) + ": " + value + ")" for key, value in outlook_folder_codes.items()])}'))
outlook = win32com.client.Dispatch(OUTLOOK+DOT+APPLICATION).GetNamespace(MAPI)

outlook_folder = outlook.GetDefaultFolder(6)
print(f'Tipo de folder: {outlook_folder}')

Tipo de folder: Bandeja de entrada


Guardar archivo de outlook

In [ ]:
importlib.reload(dmf)

# Buscar el correo más reciente con archivo Excel
mails = outlook_folder.Items

# Ordenar por fecha descendente
mails.Sort("[ReceivedTime]", True) # (mails) Es un objeto lista

''' MAIL PROPERTIES
    | (mail.Subject) (mail.ReceivedTime) (mail.SenderName)       |
    | (mail.SenderEmailAddress) (mail.To) (mail.CC)              |
    | (mail.Body) (mail.Attachments.Count) (mail.CreationTime)   |
    | (mail.LastModificationTime) (mail.EntryID)                 |
'''

(   
    df,
    conductores_ralenti['mail_file_address'], 
    conductores_ralenti['mail_received_time'], 
    conductores_ralenti['mail_data_time'], 
) = dmf.download_mail_file(
    mails, 
    outlook_object_types['MailItem'],
    project_address,
    conductores_ralenti
)

print(f'Fecha de datos: {conductores_ralenti['mail_data_time']}\n')

0
94
Fecha de datos: 21-06-2025



In [ ]:
importlib.reload(fm)
importlib.reload(pm)

df = fm.file_processing(
        df,
        conductores_ralenti,
        PEDREGAL
    )

pm.show_df(df)
pm.show_document(conductores_ralenti)

In [ ]:
importlib.reload(gs)

gs.main(project_address, df, conductores_ralenti)

### Envio de Reporte

In [ ]:
importlib.reload(srtw)

# Envio de reportes a grupo de wssp
def send_reports(group_option):
    WSSP_CONFIF = {
        'page_url': 'https://web.whatsapp.com',
        'group_names': ['PEDREGAL - DISTRIBUCIÓN'] if group_option == 1 else ['GrupoPrueba - ón'],
    }

    srtw.main(project_address, WSSP_CONFIF, conductores_ralenti['mail_data_time'])

# 🔁 Bucle de confirmación
while True:
    print("\nSelecciona grupo de wssp:")
    print("  [1] Grupo Oficial")
    print("  [2] Grupo de Prueba")
    print("  [3] Terminar proceso")

    group_option = int(input("\n>> Opción (1 - 3): "))

    if group_option == 3 or group_option > 2:
        print("\n[✓] Proceso finalizado.")
        break  # salir del bucle
    else:        
        send_reports(group_option)

### Limpiar carpeta

In [ ]:
carpeta = Path(project_address)
#print(list(carpeta.glob('*.png')))

# Eliminar archivos .png
for imagen in carpeta.glob('*.png'):
    imagen.unlink()  # .unlink() elimina el archivo
    print(f'Eliminando: {imagen}')

# Eliminar archivos .xlsx
for imagen in carpeta.glob('*.xlsx'):
    imagen.unlink()  # .unlink() elimina el archivo
    print(f'Eliminando: {imagen}')

### Export it as .py

In [1]:
import nbformat
from nbconvert import PythonExporter

# Cargar el notebook
notebook_path = "main.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Eliminar la última celda
nb['cells'] = nb['cells'][:-1]

# Convertir a script .py
exporter = PythonExporter()
codigo_python, _ = exporter.from_notebook_node(nb)

# Guardar el archivo .py
with open("main.py", "w", encoding="utf-8") as f:
    f.write(codigo_python)

print("✅ Archivo .py creado sin la última celda.")

✅ Archivo .py creado sin la última celda.
